In [1]:
# Импорт библиотек
import os
import time

import GTU
import KU_TU
import mat_properties as prop
import numpy as n
import pandas as pd
import SP
from calculate_CCGT_PKM import Calculate_CCGT_PKM_iter
from scipy.optimize import root

# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
GTU_ISO = pd.read_excel("input.xlsx", sheet_name="ISO", index_col=0)
GTU_input = pd.read_excel("input.xlsx", sheet_name="GTU_input", index_col=0)
# рабочая таблица (=номинал в 1 итерации)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)
# рабочая таблица показателей блоков
heaters = pd.read_excel("blocks.xlsx", sheet_name="heaters", index_col=0)
electric = pd.read_excel("blocks.xlsx", sheet_name="electric", index_col=0)
accumulation = pd.read_excel("blocks.xlsx", sheet_name="accumulation", index_col=0)
############################################################
# Теплосеть и перекидка температуры воздуха
gas_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
water_streams.loc["AIR", "T":"P"] = [GTU_input.loc["tair", 1], 0.1]
Tnv = gas_streams.at["AIR", "T"]
water_streams.at["SWIN", "T"] = SP.Tset(Tnv)[1]
water_streams.at["SWOUT", "T"] = SP.Tset(Tnv)[0]
water_streams.at["SWIN-TURB", "T"] = water_streams.at["SWIN", "T"]
water_streams.at["SWIN-TURB", "G"] = water_streams.at["SWIN", "G"]
water_streams.at["SP2-WOUT", "T"] = water_streams.at["SWOUT", "T"]
############################################################

# Основные эффективности оборудования
KPD_PN = 0.8074
KPD_KN = 0.75
KPD_to = 0.99
KPD_SP = 0.99


# Параметры, отвечающие за процесс расчета
Calcmethod = "hybr"
Calctolerance = 10**-2
Maxiterations_KU_TU = 8
Maxiterations_cotel = 5
Maxiterations_turbine = 15

# Параметры режима работы ПГУ
# Расчет для минимума нагрузки
Сalculate_minimum = True


# Расчет для работы с теплофикацией
# Teplo = int(False)
# Отбор пара высокго давления или доля или кг/с
steamVD_fraction_to_turbine = 1


# Состав газов в номинале в ГТУ
gasmix = "Nitrogen*Oxygen*CO2*Water*Argon"
# Считывание рефпропа
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
fractiongas0 = list(gas_streams0.loc["GTU-PEVD", "N2":"Ar"])
gas0 = prop.Materials_prop(
    gasmix,
    fractiongas0,
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    prop.REFPROP_p_rho,
    prop.REFPROP_s_q,
    RP=RP,
)
# Задание энтальпий газа в номинальном режиме
Temperatures = gas_streams0.loc["GTU-KU":"GPK-out", "T"]
Pressure = gas_streams0.loc["GTU-KU", "P"]
gas_streams0.loc["GTU-KU":"GPK-out", "H"] = list(
    map(lambda x: gas0.p_t(Pressure, x)["h"], Temperatures)
)


#####################Максимов#####################
water_streams.at["SWIN", "H"] = water.p_t(1, water_streams.at["SWIN", "T"])["h"]
water_streams.at["SWOUT", "H"] = water.p_t(1, water_streams.at["SWOUT", "T"])["h"]

# Время зарядки/разрядки, часы
time_ac = 4
# Время ожидания, часы
time_jdat = 12
# Конструкция аккумулятора
constr = {
    "Diametr": 20,
    "kolichestvo": 8,
    "Visota": 20,
    "lambda_min_vata": 0.045,
    "delta_min_vata": 0.01,
}
PKM_zaryad = True
PKM_razryad = False
syngas_streams = pd.read_excel("streams.xlsx", sheet_name="syngas", index_col=0)
##################################################


############################################################
# Задание ГТУ
Gas_turbine = GTU.gtu(GTU_ISO, "GTU-KU")

Iter_pkm = 4
pkm_pgu_tol = 10**-2

arguments_all_it = [
    Maxiterations_KU_TU,
    Maxiterations_cotel,
    Maxiterations_turbine,
    gas_streams0,
    water_streams0,
    GTU_ISO,
    GTU_input,
    gas_streams,
    water_streams,
    heaters,
    electric,
    Gas_turbine,
    gas0,
    water,
    PKM_zaryad,
    PKM_razryad,
    syngas_streams,
    Calcmethod,
    Calctolerance,
    KPD_PN,
    KPD_KN,
    KPD_to,
    KPD_SP,
    steamVD_fraction_to_turbine,
    accumulation,
    time_ac,
    constr,
    time_jdat,
]


########################ОГРАНИЧЕНИЯ НА РАБОТУ ПГУ########################
# ЕСЛИ ТЕПЛОФИКАЦИЯ (из документов по ПГУ-220Т)
# Gк_мин=4,44 кг/с вроде
# Nтурбины мин = 17,6 МВт (примерно 25%)
# Максимальноа давление в отборах СП 0,245, 0,198 МПа (Теплофикационная паровая турбина Т-63/76-8.8 для серии ПГУ-230)
# ДЛЯ ВСЕХ РЕЖИМОВ (Из Трухния по ПГУ-450Т)
# Gвд_мин=25% от номинала
# Gнд_мин = 50% от номинала


########РАСЧЕТ расхода пара на ПКМ из условия заполнения хранилища################
########РАСЧЕТ мощности ГТУ из условия тепловой мощности и ################

################Расчет минимальной нагрузки ГТУ при остальных нормальных условиях############

Max_iterations_minimum = 10

n_GTU_it = [0]
if Сalculate_minimum == True:
    gas_streams.loc["GTU-PEVD", "G"] = gas_streams.loc["GTU-KU", "G"]
    n_GTU = GTU_input.at["n", 1]
    start_time = time.time()
    Delta_min=0
    for i in range(Max_iterations_minimum):
        print("n_GTU:", n_GTU_it)
        if i < 4:
            (
                New_iterations_KU_TU,
                New_iterations_cotel,
                New_iterations_turbine,
                New_Iter_pkm,
            ) = (2, 2, 15, 3)
        else:
            (
                New_iterations_KU_TU,
                New_iterations_cotel,
                New_iterations_turbine,
                New_Iter_pkm,
            ) = (
                Maxiterations_KU_TU,
                Maxiterations_cotel,
                Maxiterations_turbine,
                Iter_pkm,
            )

        gas_streams = Calculate_CCGT_PKM_iter(
            arguments_all_it, New_Iter_pkm, pkm_pgu_tol
        )
        Delt_Gcnd = 100
        Delt_Nturb = 100
        Delt_Gcnd = (water_streams.at["INKOND", "G"] - 4.44) / 4.44
        Delt_Nturb = (electric.at["Turbine", "Ni"] - 17.6) / 17.6
        Delt_Gvd = (
            water_streams.at["PEVD-DROSVD", "G"] / water_streams0.at["PEVD-DROSVD", "G"]
            - 0.25
        ) / 0.25
        Delt_Gnd = (
            water_streams.at["PPND-DROSND", "G"] / water_streams0.at["PPND-DROSND", "G"]
            - 0.5
        ) / 0.5
        
        
        
        arguments_all_it[0], arguments_all_it[1], arguments_all_it[2] = (
            New_iterations_KU_TU,
            New_iterations_cotel,
            New_iterations_turbine,
        )
        
        Delta_n_GTU = abs((n_GTU_it[-1] - n_GTU_it[-2]) / n_GTU_it[-1] * 100)
        Delta_min = min(Delt_Gcnd, Delt_Nturb, Delt_Gvd, Delt_Gnd)
        n_GTU = n_GTU - Delta_min / 3
        GTU_input.at["n", 1] = n_GTU
        n_GTU_it.append(round(n_GTU, 5))
        
        
        print(
            f"Время {i+1} итерации расчета мощности ГТУ при ПГУ с ПКМ: --- {round((time.time() - start_time), 1)} сек. ---"
        )
        print(f"Отклонение от ограничения минимальное равно {Delta_min}")
        
        if n_GTU == 1 and Delta_min < 0:
            print("Мощность ГТУ 100% и расход пара все еще слишком мал")

        if abs(Delta_min) < Calctolerance and Delta_n_GTU < Calctolerance:
            arguments_all_it[0], arguments_all_it[1], arguments_all_it[2] = (
                Maxiterations_KU_TU,
                Maxiterations_cotel,
                Maxiterations_turbine,
            )
            n_GTU_it.append(round(n_GTU, 5))
            gas_streams = Calculate_CCGT_PKM_iter(
                arguments_all_it, New_Iter_pkm, pkm_pgu_tol
            )
            print(f"Отклонение от ограничения минимальное равно {Delta_min}")
            print(f"Относительная мощность ГТУ равна {n_GTU}")
            print(
                f"fin минимальная мощность ПГУ:--- {round((time.time() - start_time), 1)} сек. ---"
            )
            print("n_GTU_it", n_GTU_it)
            break
        if i == Max_iterations_minimum - 1:
            print(
                "Достигнуто максимальное количество итераций минимального расхода в ПГУ",
                i + 1,
            )
            print("n_GTU_it", n_GTU_it)

n_GTU: [0]
Тепла от ПКМ не хватает на теплофикацию. Расход на СП: 48.8
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 1 итерации расчета КУ+ТУ:---  10.8 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций котла-утилизатора:2
Время 2 итерации расчета КУ+ТУ:---  23.3 сек. ---
Достигнуто максимальное количество итераций давления КУ+ПТУ: 2
Pnd_it [0.75415, 0.58418, 0.51791]
Pvd_it [9.16686, 5.89065, 4.49831]
fin КУ и ТУ:--- 23.3 сек. ---
Время 1 итерации расчета КУ+ТУ с ПКМ: --- 25.7 сек. ---
Тепла от ПКМ не хватает на теплофикацию. Расход на СП: 109.1
Достигнуто максимальное количество итераций котла-утилизатора:2
Достигнуто максимальное количество итераций расхода КУ+ПТУ: 2
Время 1 итерации расчета КУ+ТУ:---  8.2 сек. ---
Давление на входе в ЦНД меньше нуля при температуре возд

In [2]:
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,527.600273,0.1,941.958209,386.959902,0.780300,0.123700,0.030100,0.059400,0.006500
REF-SMESH,800.000000,0.1,1531.123730,52.689533,0.710321,0.009967,0.090539,0.180531,0.008642
GTU-PEVD,563.529031,0.1,1012.566411,439.649435,0.771735,0.109781,0.037497,0.074225,0.006762
PEVD-IVD,484.605700,0.1,921.127073,439.649435,0.771735,0.109781,0.037497,0.074225,0.006762
IVD-EVD,266.246965,0.1,676.852991,439.649435,0.771735,0.109781,0.037497,0.074225,0.006762
EVD-PPND,203.570848,0.1,608.972434,439.649435,0.771735,0.109781,0.037497,0.074225,0.006762
PPND-IND,201.200780,0.1,606.422530,439.649435,0.771735,0.109781,0.037497,0.074225,0.006762
IND-GPK,149.570505,0.1,551.149467,439.649435,0.771735,0.109781,0.037497,0.074225,0.006762
GPK-out,98.471499,0.1,496.900123,439.649435,0.771735,0.109781,0.037497,0.074225,0.006762


In [3]:
water_streams

,T,P,H,G,S,X
AIR,-29.000000,0.100000,NaN,NaN,NaN,NaN
PEVD-DROSVD,497.819273,3.763585,3443.739271,61.719119,NaN,NaN
IVD-PEVD,256.588419,4.437727,2798.355513,61.719119,NaN,NaN
EVD-IVD,247.848335,4.437727,1075.315516,61.719119,NaN,NaN
PEN-EVD,141.099269,4.437727,596.506561,61.719119,NaN,NaN
BND-PEN,139.929741,0.360821,588.860341,61.719119,NaN,NaN
PPND-DROSND,185.366981,0.386357,2832.620888,11.373982,NaN,NaN
IND-PPND,139.929741,0.360821,2733.352060,11.373982,NaN,NaN
GPK-IND,139.929741,0.360821,593.422545,73.093102,NaN,NaN
GPK-REC,139.929741,0.360821,593.422545,72.899552,NaN,NaN


In [4]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,700.0,2.0,3687.984095,45.441820,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
COOL-HTS,450.0,2.0,3030.242194,45.441820,0.0,0.0,0.063560,0.0,0.536620,0.050040,0.325900,0.023880
HTS-HTSCOOL,275.0,2.0,2555.053441,45.441820,0.0,0.0,0.086415,0.0,0.513767,0.050042,0.348747,0.001029
HTSCOOL-Separ,100.0,2.0,791.135714,45.441820,0.0,0.0,0.086415,0.0,0.513767,0.050042,0.348747,0.001029
Separ-SGaccum,100.0,2.0,1348.041621,18.074752,0.0,0.0,0.168668,0.0,0.050949,0.097674,0.680700,0.002009
SGaccum-Separacc,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Separacc-Sepout,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sepout-COMB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Sepout-GTU,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AIR-COMP,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
electric

,Ni,N,KPDm,KPD
PEN,0.471918,0.610389,0.773143,0.575109
CVD,NaN,NaN,NaN,NaN
CND,NaN,NaN,NaN,NaN
GTU,NaN,102.691759,NaN,29.567244
DK,NaN,6.167776,NaN,NaN
Turbine,17.596783,NaN,NaN,NaN
Tots1,10.052040,NaN,NaN,0.831207
Tots2,1.862141,NaN,NaN,0.812476
Tots3,0.466281,NaN,NaN,0.754589
Tots4,5.216321,NaN,NaN,0.574402


In [6]:
heaters

,Qw,Qg,KPD
PEVD,39799.240794,40201.253327,0.99
IVD,106321.012310,107394.961929,0.99
EVD,29545.212196,29843.648682,0.99
PPND,1109.853254,1121.063893,0.99
IND,24057.762926,24300.770632,0.99
GPK,23612.186560,23850.693495,0.99
SP2,1.538811,NaN,0.99
SP1,22.864249,NaN,0.99
OD,1.701473,NaN,0.99
Strans,129819.882492,NaN,NaN


In [7]:
accumulation

,Qw,T,V
ASW,NaN,NaN,NaN
PKM,NaN,250.0,10000.0


In [8]:
PKM_zaryad = False
PKM_razryad = True

Maxiterations_KU_TU = 20
Maxiterations_cotel = 5
Maxiterations_turbine = 20

arguments_all_it = [
    Maxiterations_KU_TU,
    Maxiterations_cotel,
    Maxiterations_turbine,
    gas_streams0,
    water_streams0,
    GTU_ISO,
    GTU_input,
    gas_streams,
    water_streams,
    heaters,
    electric,
    Gas_turbine,
    gas0,
    water,
    PKM_zaryad,
    PKM_razryad,
    syngas_streams,
    Calcmethod,
    Calctolerance,
    KPD_PN,
    KPD_KN,
    KPD_to,
    KPD_SP,
    steamVD_fraction_to_turbine,
    accumulation,
    time_ac,
    constr,
    time_jdat,
]
New_Iter_pkm = 20
pkm_pgu_tol = 10**-2
CCGT = Calculate_CCGT_PKM_iter(arguments_all_it, New_Iter_pkm, pkm_pgu_tol)

Тепла в ГВТО хватает на теплофикацию
Не сходится баланс камеры сгорания 41.958396943850346 5.300648808770347e-12 41.958396943845045
Рассчитана пиковая ГТУ, расход воздуха: 932.7595236233258
Достигнуто максимальное количество итераций котла-утилизатора:3
Время 1 итерации расчета КУ+ТУ:---  27.7 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:3
Время 2 итерации расчета КУ+ТУ:---  46.2 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:3
Время 3 итерации расчета КУ+ТУ:---  66.1 сек. ---
Достигнуто максимальное количество итераций котла-утилизатора:3
Время 4 итерации расчета КУ+ТУ:---  86.6 сек. ---
Время 5 итерации расчета КУ+ТУ:---  102.3 сек. ---
Время 6 итерации расчета КУ+ТУ:---  115.3 сек. ---
Время 7 итерации расчета КУ+ТУ:---  131.1 сек. ---
Время 8 итерации расчета КУ+ТУ:---  143.6 сек. ---
Время 9 итерации расчета КУ+ТУ:---  153.7 сек. ---
Время 10 итерации расчета КУ+ТУ:---  163.4 сек. ---
Время 11 итерации расчета КУ+ТУ:---  169.5 сек.

In [9]:
gas_streams

,T,P,H,G,N2,O2,CO2,H2O,Ar
AIR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GTU-KU,527.600273,0.1,941.958209,386.959902,0.780300,0.123700,0.030100,0.059400,0.006500
REF-SMESH,800.000000,0.1,1531.123730,52.689533,0.710321,0.009967,0.090539,0.180531,0.008642
GTU-PEVD,563.529031,0.1,1012.566411,439.649435,0.771735,0.109781,0.037497,0.074225,0.006762
PEVD-IVD,484.605700,0.1,921.127073,439.649435,0.771735,0.109781,0.037497,0.074225,0.006762
IVD-EVD,266.246965,0.1,676.852991,439.649435,0.771735,0.109781,0.037497,0.074225,0.006762
EVD-PPND,203.570848,0.1,608.972434,439.649435,0.771735,0.109781,0.037497,0.074225,0.006762
PPND-IND,201.200780,0.1,606.422530,439.649435,0.771735,0.109781,0.037497,0.074225,0.006762
IND-GPK,149.570505,0.1,551.149467,439.649435,0.771735,0.109781,0.037497,0.074225,0.006762
GPK-out,98.471499,0.1,496.900123,439.649435,0.771735,0.109781,0.037497,0.074225,0.006762


In [10]:
water_streams

,T,P,H,G,S,X
AIR,-29.000000,0.100000,NaN,NaN,NaN,NaN
PEVD-DROSVD,505.970986,7.932431,3415.042369,46.179701,NaN,NaN
IVD-PEVD,296.120423,8.128274,2756.761659,46.179701,NaN,NaN
EVD-IVD,295.864157,8.128274,1321.994055,46.179701,NaN,NaN
PEN-EVD,186.231713,8.128274,794.031780,46.179701,NaN,NaN
BND-PEN,161.458130,0.641504,681.811714,46.179701,NaN,NaN
PPND-DROSND,206.863574,0.636990,2863.747499,12.985249,NaN,NaN
IND-PPND,161.458129,0.641504,2759.034488,12.985249,NaN,NaN
GPK-IND,158.208483,0.641504,667.725226,59.164950,NaN,NaN
GPK-REC,158.208483,0.641504,667.725226,70.904468,NaN,NaN


In [11]:
syngas_streams

,T,P,H,G,N2,O2,CO2,Ar,H2O,CH4,H2,CO
REF-COOL,700.000000,2.0,3687.984095,45.441820,0.000000,0.000000,0.063560,0.000000,0.536620,0.050040,0.325900,0.023880
COOL-HTS,450.000000,2.0,3030.242194,45.441820,0.000000,0.000000,0.063560,0.000000,0.536620,0.050040,0.325900,0.023880
HTS-HTSCOOL,275.000000,2.0,2555.053441,45.441820,0.000000,0.000000,0.086415,0.000000,0.513767,0.050042,0.348747,0.001029
HTSCOOL-Separ,100.000000,2.0,791.135714,45.441820,0.000000,0.000000,0.086415,0.000000,0.513767,0.050042,0.348747,0.001029
Separ-SGaccum,100.000000,2.0,1348.041621,18.074752,0.000000,0.000000,0.168668,0.000000,0.050949,0.097674,0.680700,0.002009
SGaccum-Separacc,-21.803926,2.0,998.444452,365775.995050,0.000000,0.000000,0.168738,0.000000,0.050357,0.097508,0.681387,0.002009
Separacc-Sepout,-21.803926,2.0,879.636427,336452.977654,0.000000,0.000000,0.177686,0.000000,0.000000,0.102679,0.717520,0.002115
Sepout-COMB,-21.803926,2.0,879.636427,22.283679,0.000000,0.000000,0.177686,0.000000,0.000000,0.102679,0.717520,0.002115
Sepout-GTU,-21.803926,2.0,879.636427,1.081111,0.000000,0.000000,0.177686,0.000000,0.000000,0.102679,0.717520,0.002115
AIR-COMP,15.000000,0.1,293.187169,932.759524,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
accumulation

,Qw,T,V
ASW,NaN,NaN,NaN
PKM,8.915339e+09,-21.803926,10000.0
